#Requirements

# 

---

## Exercise 1

As part of this exercise we will be performing two operations on a large dataset using Numpy. First, considering an array of values, we will compute the percentage differential from one value to the next, as in the example below.
``` example
    input_array = [43, 63, 72, 64, 16]
    percent_diff = [(43-63)/43, (63-72)/63, (72-64)/72, (64-16)/64]
```
Second, we will compute the cummulative sum on an input array:
``` example
    input_array = [43, 63, 72, 64, 16]
    cumsum = [43, 43+63, 43+63+72, 43+63+72+64, 43+63+72+64+16]
```
Working code for both functions is provided in the following cells, the task is to re-write both functions to accelerate the computation. Several means of speeding up code have been covered . Those include the use of caching, the use of native languages, the use of parallel computing, or the modification of the default algorithm. You can use one or several combined strategies.

---

The following cell's code is from [https://github.com/rakshitha123/TSForecasting/blob/master/utils/data_loader.py] and enables the convert an input dataset in .tsf format into a pandas data frame.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numba import njit, prange

# https://github.com/rakshitha123/TSForecasting/blob/master/utils/data_loader.py

from datetime import datetime
from distutils.util import strtobool


# Converts the contents in a .tsf file into a dataframe and returns it along
# with other meta-data of the dataset: frequency, horizon, whether the dataset
# contains missing values and whether the series have equal lengths
#
# Parameters
# full_file_path_and_name - complete .tsf file path
# replace_missing_vals_with - a term to indicate the missing values in series
# in the returning dataframe
# value_column_name - Any name that is preferred to have as the name of the
# column containing series values in the returning dataframe
def convert_tsf_to_dataframe(
    full_file_path_and_name,
    replace_missing_vals_with="NaN",
    value_column_name="series_value",
):
    col_names = []
    col_types = []
    all_data = {}
    line_count = 0
    frequency = None
    forecast_horizon = None
    contain_missing_values = None
    contain_equal_length = None
    found_data_tag = False
    found_data_section = False
    started_reading_data_section = False

    with open(full_file_path_and_name, "r") as file:  # , encoding="cp1252"
        for line in file:
            # Strip white space from start/end of line
            line = line.strip()

            if line:
                if line.startswith("@"):  # Read meta-data
                    if not line.startswith("@data"):
                        line_content = line.split(" ")
                        if line.startswith("@attribute"):
                            if len(line_content) != 3:  # Attributes have both name and type
                                raise Exception("Invalid meta-data specification.")

                            col_names.append(line_content[1])
                            col_types.append(line_content[2])
                        else:
                            if len(line_content) != 2:  # Other meta-data have only values
                                raise Exception("Invalid meta-data specification.")

                            if line.startswith("@frequency"):
                                frequency = line_content[1]
                            elif line.startswith("@horizon"):
                                forecast_horizon = int(line_content[1])
                            elif line.startswith("@missing"):
                                contain_missing_values = bool(strtobool(line_content[1]))
                            elif line.startswith("@equallength"):
                                contain_equal_length = bool(strtobool(line_content[1]))

                    else:
                        if len(col_names) == 0:
                            raise Exception("Missing attribute section. "
                                            "Attribute section must come before data.")

                        found_data_tag = True
                elif not line.startswith("#"):
                    if len(col_names) == 0:
                        raise Exception("Missing attribute section. "
                                        "Attribute section must come before data.")
                    elif not found_data_tag:
                        raise Exception("Missing @data tag.")
                    else:
                        if not started_reading_data_section:
                            started_reading_data_section = True
                            found_data_section = True
                            all_series = []

                            for col in col_names:
                                all_data[col] = []

                        full_info = line.split(":")

                        if len(full_info) != (len(col_names) + 1):
                            raise Exception("Missing attributes/values in series.")

                        series = full_info[len(full_info) - 1]
                        series = series.split(",")

                        if len(series) == 0:
                            raise Exception(
                                "A given series should contains a set of comma separated "
                                "numeric values. At least one numeric value should be there "
                                "in a series. Missing values should be indicated with ? symbol"
                            )

                        numeric_series = []

                        for val in series:
                            if val == "?":
                                numeric_series.append(replace_missing_vals_with)
                            else:
                                numeric_series.append(float(val))

                        if numeric_series.count(replace_missing_vals_with) == len(numeric_series):
                            raise Exception(
                                "All series values are missing. A given series should contains "
                                "a set of comma separated numeric values. At least one numeric "
                                "value should be there in a series."
                            )

                        all_series.append(pd.Series(numeric_series).array)

                        for i in range(len(col_names)):
                            att_val = None
                            if col_types[i] == "numeric":
                                att_val = int(full_info[i])
                            elif col_types[i] == "string":
                                att_val = str(full_info[i])
                            elif col_types[i] == "date":
                                att_val = datetime.strptime(full_info[i], "%Y-%m-%d %H-%M-%S")
                            else:
                                raise Exception(
                                    "Invalid attribute type."
                                )   # Currently, the code supports only numeric, string and date types.
                                    # Extend this as required.

                            if att_val is None:
                                raise Exception("Invalid attribute value.")
                            else:
                                all_data[col_names[i]].append(att_val)

                line_count = line_count + 1

        if line_count == 0:
            raise Exception("Empty file.")
        if len(col_names) == 0:
            raise Exception("Missing attribute section.")
        if not found_data_section:
            raise Exception("Missing series information under data section.")

        all_data[value_column_name] = all_series
        loaded_data = pd.DataFrame(all_data)

        return (
            loaded_data,
            frequency,
            forecast_horizon,
            contain_missing_values,
            contain_equal_length,
        )


# Example of usage
# loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe("TSForecasting/tsf_data/sample.tsf")

# print(loaded_data)
# print(frequency)
# print(forecast_horizon)
# print(contain_missing_values)
# print(contain_equal_length)

The following cell loads a dataset downloaded from [https://zenodo.org/record/7371038].
The following description is from the dataset webpage:

"This dataset contains 145063 time series representing the number of hits or web traffic for a set of Wikipedia pages from 2015-07-01 to 2022-06-30. This is an extended version of the dataset that was used in the Kaggle Wikipedia Web Traffic forecasting competition. For consistency, the same Wikipedia pages that were used in the competition have been used in this dataset as well. The colons (:) in article names have been replaced by dashes (-) to make the .tsf file readable using our data loaders.

The original dataset contains missing values. They have been simply replaced by zeros.

The data were downloaded from the Wikimedia REST API. According to the conditions of the API, this dataset is licensed under CC-BY-SA 3.0 and GFDL licenses."

You can download the dataset from [https://zenodo.org/record/7371038/files/web_traffic_extended_dataset_without_missing_values.zip]. Note that the file is large: 433 MB. The cells convert the dataset into a Numpy compressed file for easier use in the remainder of this jupyter notebook.

In [ ]:
# https://zenodo.org/record/7371038
df, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe(
    "web_traffic_extended_dataset_without_missing_values.tsf"
)
hits_arr = np.stack(df["series_value"].apply(np.asarray).tolist())
np.savez_compressed("web_traffic_extended_dataset_without_missing_values", hits_arr)

In [ ]:
# Reload the dataset and extract a single Serie
hits_arr = np.load("web_traffic_extended_dataset_without_missing_values.npz")["arr_0"]

The following cell implement the `pct_change1` function, used to compute the percentage difference change beetween succesive values, as aforementioned. The cell is also used to time the function and display the result as a figure.

In [ ]:
def pct_change1(n):
    result = np.diff(n) / n[:, :-1]
    result[np.isnan(result)] = 0  # difference is 0 and both values are 0
    result[np.isinf(result)] = 1  # first value was 0 and next is not 0
    return result


pct = pct_change1(hits_arr)

%timeit pct_change1(hits_arr)

plt.figure(figsize=(14, 5))
plt.plot(pct[10559, 300:800])

### Question 1  

Modify the following cell to implement the function `pct_change2` to accelerate the computation. The `np.allclose()` function checks that the results obtained using `pct_change1` and `pct_change2` are near identical.

In [ ]:
def pct_change2(n):
    result = None
    #TODO
    return result


pct_n = pct_change2(hits_arr)

%timeit pct_change2(hits_arr)

print(np.allclose(pct, pct_n))

The `cumsum` function from numpy can be used to compute the cummulative sum of the `hits_arr` array, as shown below.

In [ ]:
cs1 = np.cumsum(hits_arr, 1)

%timeit np.cumsum(hits_arr,1)

### Question 2  

Modify the following cell to implement the function `cumsum2` to accelerate the computation. The `np.allclose()` function enables to check that the results obtained using `np.cumsum` and `cumsum2` are similar.

In [ ]:
def cumsum2(n):
    result = None
    #TODO
    return result


cs2 = cumsum2(hits_arr)

%timeit cumsum2(hits_arr)

np.allclose(cs1, cs2)

---

## Exercise 2

The aim of the exercise is to develop a effective solution to solve the [travelling salesman problem](https://en.wikipedia.org/wiki/Travelling_salesman_problem).

The idea of the Travelling Salesman Problem is to find the shortest path necessary to visit all locations and return to the start. For an example of visiting four locations, cycle length is the distance travelled from starting location A to location B, then to location C, then to location D, then back to A. The brute force approach to solving this is consider every possible cycle between the locations, however for N locations there are N! possible cycles so this isn't really tractable for anything above small values like 10.

The following cell generate a random graph with 100 nodes, links the nodes in indices increasing order (to create edges between pair of nodes) and display the resulting solution.

In [ ]:
from functools import partial
from itertools import permutations
from timeit import timeit

import matplotlib.pyplot as plt
import numpy as np
from numba import njit, prange


def create_locations(num, width):
    return np.random.uniform(0, width, (num, 2))

def plot_cycle(locs, cycle):
    x, y = zip(*locs)
    plt.figure()
    plt.scatter(x, y)

    for idx in range(cycle.shape[0]):
        x0, y0 = locs[cycle[idx]]
        x1, y1 = locs[cycle[(idx + 1) % locs.shape[0]]]
        plt.plot([x0, x1], [y0, y1], c="r",linewidth=1)

locs = create_locations(100, 10)
cycle = np.arange(len(locs))
plot_cycle(locs, cycle)

In order to optimise the traveling salesman problem, one need to compute the overall length of the current solution. The following cell provide a numpy based implementation in the `cycle_length_numpy` function.

In [ ]:
def cycle_length_numpy(locs,cycle):
    locs_ordered=locs[cycle]
    return np.linalg.norm(locs_ordered - np.roll(locs_ordered, 1, 0), axis=1).sum()


print(cycle_length_numpy(locs,cycle))
%timeit cycle_length_numpy(locs,cycle)

### Question 3 

Using the techniques from the course implement an optimised version of the algorithm, assuming your input array of locations has shape `(N, 2)` (ie. `N` 2D coordinates) and array of cycle indices has shape `(N,)`. It's possible to achieve of speedup of at least 10 times over the algorithm given above without using threading/multiprocessing so do not use these techniques.

Using Numpy requires the creation of a number of extra arrays used for calculation, however an optimised one can be implemented which doesn't need to create any additional arrays but instead accumulates a result value. You should consider this in your implementation and you will be marked on whether you can avoid this extra allocation.

You are expected to provide a solution to the `cycle_length` function in the following cell. The `cycle_length` function should be optimised in term of computation time when compare to the numpy `cycle_length_numpy` function.

In [ ]:
def cycle_length_fast(locs, cycle):
    """Compute the length of cycle given by the indices `cycle` and positions `locs`."""
    length = None
    # TODO
    return length


print(cycle_length_fast(locs, cycle))
%timeit cycle_length_fast(locs,cycle)

### Question 4 

Reimplement your algorithm from above to use threading and see how it's performance is relative to the fast and Numpy versions. 

In [ ]:
def cycle_length_thread(locs, cycle):
    """Compute the length of cycle given by the indices `cycle` and positions `locs`."""
    length = None
    # TODO
    return length


print(cycle_length_thread(locs, cycle))
%timeit cycle_length_thread(locs, cycle)

### Question 5 

Times for runs of the three versions of the algorithm are calculated below and plotted. At certain points one version of the algorithm or another becomes the fastest, why is this? Specifically at what point does the threaded version become fastest and why?

In [ ]:
loc_sizes = [10, 100, 1_000, 10_000, 100_000, 1_000_000]
locs = create_locations(loc_sizes[-1], 10)
cycle = np.arange(loc_sizes[-1])

numpy_times = [timeit(lambda: cycle_length_numpy(locs, cycle[:n]), number=20) for n in loc_sizes]
fast_times = [timeit(lambda: cycle_length_fast(locs, cycle[:n]), number=20) for n in loc_sizes]
thread_times = [timeit(lambda: cycle_length_thread(locs, cycle[:n]), number=20) for n in loc_sizes]

plt.loglog(loc_sizes, numpy_times, label="Numpy")
plt.loglog(loc_sizes, fast_times, label="Fast")
plt.loglog(loc_sizes, thread_times, label="Thread")
plt.legend()

Answer the previous question in this cell [200 word answer max].

*YOUR ANSWER HERE*

#### Finding a Good Cycle

A number of approaches exist to find a good cycle for non-trivial location sizes. There are exact algorithms that are faster than brute force but are still NP-hard. A approximate solution involves some technique to find a good solution in some defined manner that's better than just chance. The implementation below uses a genetic algorithm approach to start from an initial cycle (parent) and modify parts of it in a randomised way a set number of times. From the generated cycles (offspring) the shortest is chosen to be the parent of the next generation. This has similarities to genetic mutations in real organisms but simplifies the concepts immensely but allows one to start from one solution and attempt to find a nearby better solution, ultimately with the hope of converging onto a final solution that's close to an actual optimum. This approach is relatively simple and attractive for attacking real-world problems with intractable algorithmic solutions. See: https://en.wikipedia.org/wiki/Genetic_algorithm

Below is the brute force solution to the problem, don't run this with locations counts above 10 because it will take enormous amounts of time to finish:

In [ ]:
def find_best_cycle_bf(locs):
    """Try all possible cycles for the given locations, this will take longer than the age of the universe for even trivial sets."""
    nlocs = len(locs)
    indices = np.arange(nlocs)
    min_len = float("inf")
    result = None

    for p in permutations(np.arange(nlocs), nlocs):
        indices[:] = p
        # modify the cycle length to use your fastest implementation
        clen = cycle_length_numpy(locs, indices)
        if clen < min_len:
            min_len = clen
            result = p

    return result

test_locs = create_locations(10, 10)
best = find_best_cycle_bf(test_locs)
print(cycle_length_numpy(test_locs, np.array(best)))
plot_cycle(test_locs, np.array(best))

### Question 6 

Below is the solution using a genetic algorithm approach. For a set number of iterations, a starting cycle is mutated a randomised number of times to produce a number of offspring. From these the shortest path is chosen as the new parent and all others go extinct. 

The following function is used to mutate a given cycle. Optimise this using the techniques discussed in the course. Consider what it's doing as the form of mutation it applies and implement some additional operation that gives a better result. 

In [ ]:
def generate_mutation(cycle, num_changes):
    """Given a cycle, apply `num_changes` mutations to the cycle and return the mutant."""
    cycle = cycle.copy()

    for _ in range(np.random.randint(1, num_changes)):
        i,j = np.random.randint(0, len(cycle),(2,))
        cycle[i], cycle[j] = cycle[j], cycle[i]

    return cycle

generate_mutation(cycle,10)

%timeit generate_mutation(cycle,10)

In [ ]:
def generate_mutation_fast(cycle, num_changes):
    """Given a cycle, apply `num_changes` mutations to the cycle and return the mutant."""
    cycle = None
    # TODO
    return cycle

generate_mutation_fast(cycle,10)

%timeit generate_mutation_fast(cycle,10)

In [ ]:
def find_better_cycle(locs, init_cycle, num_iterations, num_offspring, num_changes):
    current_cycle = init_cycle
    iterlengths = []

    for _ in range(num_iterations):
        # modify the generate_mutation to use your fastest implementation
        offspring = [generate_mutation(current_cycle, num_changes) for _ in range(num_offspring)]
        family = [current_cycle] + offspring

        # modify the cycle length to use your fastest implementation
        lengths = np.array([cycle_length_numpy(locs, c) for c in family])

        idx = min(range(len(lengths)), key=lengths.__getitem__)

        current_cycle = family[idx]
        iterlengths.append(lengths[idx])

    return current_cycle, iterlengths

new_cycle=None
new_lengths=None

def _timeit_func():
    global new_cycle
    global new_lengths
    
    new_cycle, new_lengths = find_better_cycle(
        locs=locs,
        init_cycle=cycle,
        num_iterations=20000,
        num_offspring=100,
        num_changes=40
    )
    
    
tval=timeit(_timeit_func,number=1,globals=globals())

print(f"Time: {tval}, Best length: {new_lengths[-1]}")

plt.semilogy(new_lengths)
plot_cycle(locs, new_cycle)

### Question 7 

Modify the cell below (copy/pasted from cell above) to improve the result (shortest path) of the retrieved solution and/or improve the overall computation time.

In [ ]:
def find_better_cycle(locs, init_cycle, num_iterations, num_offspring, num_changes):
    current_cycle = init_cycle
    iterlengths = []

    for _ in range(num_iterations):
        # modify the generate_mutation to use your fastest implementation
        offspring = [generate_mutation(current_cycle, num_changes) for _ in range(num_offspring)]
        family = [current_cycle] + offspring

        # modify the cycle length to use your fastest implementation
        lengths = np.array([cycle_length_numpy(locs, c) for c in family])

        idx = min(range(len(lengths)), key=lengths.__getitem__)

        current_cycle = family[idx]
        iterlengths.append(lengths[idx])

    return current_cycle, iterlengths

new_cycle=None
new_lengths=None

def _timeit_func():
    global new_cycle
    global new_lengths
    
    new_cycle, new_lengths = find_better_cycle(
        locs=locs,
        init_cycle=cycle,
        num_iterations=20000,
        num_offspring=100,
        num_changes=40
    )
    
    
tval=timeit(_timeit_func,number=1,globals=globals())

print(f"Time: {tval}, Best length: {new_lengths[-1]}")

plt.semilogy(new_lengths)
plot_cycle(locs, new_cycle)

## Overall:
While writing your code and comments, you are expected to adhere to Python
style guideline (see [1]). Your code will be assessed with static and style
analysis tools  to detect any potential
defects.

Your code is expected to use features of the python language
For example, you should use iterator, generator or list
comprehension [2]  instead of `for` loop when appropriate.

You are expected to comment your code. Ensure that your
comments are meaningful. You can refer to [3] for recommendations about
good commenting practices.




## Reference:

[1] https://www.python.org/dev/peps/pep-0008

[2] https://www.w3schools.com/python/python_lists_comprehension.asp

[3] https://realpython.com/python-comments-guide/


